In [1]:
import os
import pandas as pd
import razdel
import re
import random
import json
from tqdm.auto import tqdm, trange

## Load sentence-parallel data

In [2]:
bible = pd.read_csv('erzya_bible.tsv', sep='\t')
print(bible.shape)
bible = bible.dropna()
print(bible.shape)
bible = bible[~bible.myv.str.startswith('Глава ')]
print(bible.shape)
print(bible.columns)

(12926, 4)
(12899, 4)
(12483, 4)
Index(['Unnamed: 0', 'myv', 'ru', 'source'], dtype='object')


In [3]:
def replace_brackets(text):
    return re.sub('[\]\[]', '', text)

In [4]:
bible_pairs = list(zip(bible.myv.apply(replace_brackets), bible.ru.apply(replace_brackets)))
print(len(bible_pairs))
random.choice(bible_pairs)

12483


('Сон пандсы тенст секе зыянсонть, конань сынь теизь,\n\xa0\xa0\xa0ды сынсест кежей тевсэст маштнесынзе сынст.\n\xa0\xa0\xa0Азорось, минек Пазонок, маштнесынзе сынст.Шнамо',
 'и обратит на них беззаконие их, и злодейством их истребит их, истребит их Господь Бог наш.')

In [5]:
with open('opus_aligned/wikimedia.myv-ru.myv', 'r') as f:
    opus_m = [line.strip() for line in f.readlines()]
with open('opus_aligned/wikimedia.myv-ru.ru', 'r') as f:
    opus_r = [line.strip() for line in f.readlines()]
assert len(opus_r) == len(opus_m)

In [6]:
opus_pairs = list(zip(opus_m, opus_r))
print(len(opus_pairs))

3202


In [7]:
random.choice(opus_pairs)

('Лемесь рузокс Лемесь эстэкс Таркась Путомазо Покшолмась, вп² Координатат Карула Karula rahvuspark Вырумаа, Варсту, Вана-Рооса 1979-09-23 123.6 57°42′25″ с. ш.',
 'Название на русском Название на эстонском Место Основано Площадь, км² Координаты Карула Karula rahvuspark Вырумаа, Варсту, Вана-Рооса 1979-09-23 123.6 57°42′25″ с. ш.')

In [8]:
from ast import literal_eval

In [9]:
dict1 = pd.read_csv('russian-moksha-erzya-dictionary_fixed.tsv', sep='\t')
dict1['mo'] = dict1['mo'].apply(literal_eval)
dict1['er'] = dict1['er'].apply(literal_eval)

In [10]:
dict1.sample(5)

,Unnamed: 0,ru,mo,er,ru_fix
5660,5660,недуг,[урма],[орма],недуг
2303,2303,ёлочный,[кузонь],[кузонь],ёлочный
9677,9677,укол,"[сялгофкс, пупафкс]","[сялговкс, пупавкс]",укол
9735,9735,умыться,[штамс],[шлямс],умыться
3209,3209,здоровый,"[шумбра, вии, пара, цебярь ! Уледа шумбрат!]","[шумбра, виев, паро, лезэв ! Уледе шумбрат!]",здоровый


In [11]:
dict1_pairs = []
for row in dict1.itertuples():
    for er in row.er:
        dict1_pairs.append((er, row.ru_fix))

In [12]:
print(len(dict1_pairs))
random.choice(dict1_pairs)

22499


('ёмамс', 'погибнуть')

In [13]:
joint_phrasebook = pd.read_csv('other_sources/joint_phrasebook.tsv', sep='\t')

In [15]:
joint_phrasebook.sample(10)

,myv,ru,source
12530,колавтозь,испорченный,marlamuter
6079,"бути молян, сестэ кандан","если пойду, то принесу",marlamuter
17589,организационной,организационный,marlamuter
42899,сёрмадомс кедьсэ,от,schankina
920,нолдамс,отпускать,mordovians
26545,шалакадомс,зашуметь,marlamuter
49753,нусманя,угрюмый,schankina
35917,сюронь,зерновой,schankina
7540,веялка,веялка,marlamuter
27576,эскелявтомс,заставить шагать,marlamuter


In [16]:
joint_phrasebook.isnull().sum()

myv        2
ru        35
source     0
dtype: int64

In [17]:
joint_phrasebook_pairs = [(row.myv, row.ru) for row in joint_phrasebook.dropna().itertuples() if row.myv != row.ru]
print(len(joint_phrasebook_pairs))

47860


In [33]:
with open('other_sources/sheyanova_sentence_aligned.json', 'r') as f:
    sheyanova_aligned = json.load(f)
    
sheyanova_pairs = [(p['myv'], p['ru']) for p in sheyanova_aligned]
print(len(sheyanova_pairs))

1023


In [34]:
random.choice(sheyanova_pairs)

('Сестэ тейтересь мери: — Аштек, мон вансынь ды ёвтан.',
 'Долго проливала мать горькие слезы, узнав о судьбе сына.')

In [35]:
with open('other_sources/brizhinskiy_sentence_aligned.json', 'r') as f:
    brizhinskiy_aligned = json.load(f)
    
brizhinskiy_pairs = [(p['myv'], p['ru']) for p in brizhinskiy_aligned]
print(len(brizhinskiy_pairs))

4203


In [36]:
random.choice(brizhinskiy_pairs)

('Псаканть арась мелезэ стямс, сэстав тни, конясынзе сельмензэ, невти, буто таго матедевсь.',
 'Кот не хочет вставать, потягивается, закрывает глаза, делает вид, что снова заснул.')

In [37]:
with open('other_sources/evsenyev_stories_aligned.json', 'r') as f:
    evs_pairs = json.load(f)
evs_pairs = [tuple(p) for p in evs_pairs]
print(len(evs_pairs))

3776


In [38]:
random.choice(evs_pairs)

('— Азё, мери, цёрынем, батман сал рамак!',
 '— Иди,— говорит,— сыночек, батман1 соли купи.')

In [39]:
with open('wiki/myv_ru_aligned.json', 'r') as f:
    wiki_pairs = json.load(f)
wiki_pairs = [tuple(p) for p in wiki_pairs]
print(len(wiki_pairs))

11479


In [40]:
random.choice(wiki_pairs)

('Сехте покш лейть - Вятка, Просница, Чепца.',
 'Основные реки — Вятка, Просница, Чепца.')

In [41]:
with open('other_sources/livejournal_myv_ru_aligned.json', 'r') as f:
    lj_pairs = json.load(f)
lj_pairs = [tuple(p) for p in lj_pairs]
print(len(lj_pairs))

1799


In [42]:
random.choice(lj_pairs)

('Арнема(арне, арневикс, арнемс)-мелень явома ды путома, кода Пазтнене, Покштятнене истя ломантнене.',
 'Уважение, почитание, как Творца, его держателей, Предков так и старших, просто людей и природы.')

In [43]:
with open('other_sources/rus4all_myv_ru_aligned.json', 'r') as f:
    rus4all_pairs = json.load(f)
rus4all_pairs = [tuple(p) for p in rus4all_pairs]
print(len(rus4all_pairs))

916


In [44]:
random.choice(rus4all_pairs)

('— Недляшка теденть мейле райотделэв эзинь яка, ормалгадозекс прям ёвтыя.',
 '— Неделю после этого в райотделе не появлялся, сказался больным.')

In [45]:
with open('other_sources/ussr_constitution_aligned.json', 'r') as f:
    const_pairs = json.load(f)
const_pairs = [tuple(p) for p in const_pairs]
print(len(const_pairs))

304


In [46]:
random.choice(const_pairs)

('Весе Народной Комиссариатнэнь ды сыненст подведомственной учреждениятнень, истя жо должностной башка ломантнень, истя жо СССР-нь граждантнэнь ендо законтнэнь точнойстэ топавтомаст мельга высшей надзорось путови СССР-нь Прокуроронть лангс.',
 'Высший надзор за точным исполнением законов всеми Народными Комиссариатами и подведомственными им учреждениями, равно как отдельными должностными лицами, а также гражданами СССР возлагается на Прокурора СССР.')

In [47]:
all_sources = [
    dict1_pairs,
    joint_phrasebook_pairs ,
    opus_pairs,
    bible_pairs ,
    sheyanova_pairs, 
    brizhinskiy_pairs,
    wiki_pairs,
    lj_pairs,
    rus4all_pairs,
    const_pairs,
    evs_pairs,
]

In [48]:
def dashrepl(matchobj):
    return matchobj.group(1) + matchobj.group(3)

def fix_breaks(text):
    return re.sub('([а-яА-ЯёЁ])([\-+]\s+)([а-яА-ЯёЁ])', dashrepl, text)

In [49]:
def fix_text(text):
    text = re.sub('\s+', ' ', text).strip()
    text = fix_breaks(text)
    return text

In [50]:
for s in all_sources:
    for i, [t1, t2] in enumerate(s):
        s[i] = (fix_text(t1), fix_text(t2))

In [51]:
for i, s in enumerate(all_sources):
    set(s)

In [52]:
all_pairs = sorted({
    c for s in all_sources for c in s 
    if c[0] and c[1]
})
print(len(all_pairs))

77503


In [300]:
for _ in range(10):
    print(random.choice(all_pairs))

('зняро ашти те скалось?', 'сколько стоит эта корова?')
('аней', 'солнечный')
('1957-це иестэ прядызе Гнесиных лемсэ Седямонь училищанть, 1972 иестэ тесэ тонавтсь (1983 иестэ — профессор, 1980—1985 — сольной морамонь кафедрань прявтветий.', 'С 1972 года преподавала в Музыкально-педагогическом институте им. Гнесиных (с 1983 — профессор, в 1980—1985 — заведующая кафедрой сольного пения).')
('тарксема', 'привлечение')
('действующей армия', 'действующая армия')
('Кинь ваномо эно якиде? Пазмарицянь? Виде, Иоанн пазмариця, ялатеке кортан тенк: сон пазмарицядояк покш.', 'Что же смотреть ходили вы? пророка ли? Да, говорю вам, и больше пророка.')
('кедьге', 'тара')
('Леенть лангсо аштить: Мордвиновка, Васильевка, Родионовка, Артюховка, Айдарали, Тятер-Арасланово, Зерикло, Зобово, Нижние Кузлы, Софиевка.', 'Населённые пункты у реки: Мордвиновка, Васильевка, Родионовка, Артюховка, Айдарали, Тятер-Арасланово, Зерикло, Зобово, Нижние Кузлы, Софиевка.')
('алкине', 'неглубокий')
('манчемс', 'лукавить

# Select traindev data

In [319]:
print(len(set(bible_pairs)))

random.seed(1)
bible_devtest = random.sample(sorted(set(bible_pairs)), k=800)

11913


In [320]:
print(len(set(sheyanova_pairs)))

random.seed(1)
tales_devtest = random.sample(sorted(set(sheyanova_pairs)), k=200)

1021


In [321]:
print(len(set(const_pairs)))

random.seed(1)
const_devtest = random.sample(sorted(set(const_pairs)), k=100)

295


In [322]:
print(len(set(brizhinskiy_pairs)))

random.seed(1)
games_devtest = random.sample(sorted(set(brizhinskiy_pairs)), k=500)

4010


In [323]:
print(len(set(rus4all_pairs)))

random.seed(1)
fiction_devtest = random.sample(sorted(set(rus4all_pairs)), k=200)

890


In [324]:
print(len(set(wiki_pairs)))

random.seed(1)
wiki_devtest = random.sample(sorted(set(wiki_pairs)), k=1200)

10089


In [328]:
components = {
    'bible': bible_devtest,
    'tales': tales_devtest,
    'constitution': const_devtest,
    'games': games_devtest,
    'fiction': fiction_devtest,
    'wiki': wiki_devtest
}
dev = {k: v[:len(v)//2] for k, v in components.items()}
test = {k: v[len(v)//2:] for k, v in components.items()}

In [329]:
print(len({s for c in dev.values() for s in c}))
print(len({s for c in test.values() for s in c}))

1500
1500


In [331]:
with open('corpus/dev1000.v1.json', 'w') as f:
    json.dump(dev, f, indent=2, ensure_ascii=False)

In [332]:
with open('corpus/test1000.v1.json', 'w') as f:
    json.dump(test, f, indent=2, ensure_ascii=False)

In [582]:
devtest = {s for c in components.values() for s in c}
len(devtest)

3000

In [584]:
devtest_er = {s[0] for c in components.values() for s in c}
len(devtest_er)

2994

In [586]:
train = sorted(set(all_pairs).difference(devtest))
print(len(train))

74503


In [587]:
with open('corpus/train.v1.json', 'w') as f:
    json.dump(train, f, indent=2, ensure_ascii=False)

# Prepare the whole crawled corpus

In [336]:
import os
import pandas as pd
import razdel
import re
import random
import json
from tqdm.auto import tqdm, trange

In [390]:
bp = 'books_mono/'
books_sents = []
for fn in os.listdir(bp):
    if fn.endswith('.txt'):
        print(fn)
        with open(bp + fn, 'r') as f:
            raw_lines = f.readlines()
        sents = []
        raw_text = ''.join(raw_lines).replace('\xa0', ' ')
        raw_text = re.sub('\s+', ' ', raw_text).strip().replace('* ', '')
        
        for sent in list(razdel.sentenize(raw_text)):
            text = sent.text.replace('-\n', '').replace('\n', ' ').strip()
            if len(text) >= 3 and len(text) <= 500:
                sents.append(fix_text(text))
        books_sents.extend(sents)
        
print(len(books_sents))

Balankin_001.txt
BeBANMaksimAfanas'evich_002.txt
BeZBoRoDoVMihailIl'ich_002.txt
EjovAnatoliiiFedorovich_002.txt
GolenkovNikolaiiBorisovich_002.txt
Kemaiikina_001.txt
kosheleva_001.txt
KyldyrkaevEAkovEAkovlevich_001.txt
KyldyrkaevEAkovEAkovlevich_002.txt
LiyleakinaSerafimaMarkovna_001.txt
RadaevVasiliiiKyz'mich_002.txt
Александр Арапов.txt
Лобанов.txt
радаев.txt
Яков Кулдуркаев - ЭРЬМЕЗЬ.txt
52870


In [395]:
random.choice(books_sents)

'— Оргодсь кенкшка комадо.'

In [397]:
bp = 'fennougrica_text/'
fennougrica_sents = []

for fn in os.listdir(bp):
    if fn.endswith('.txt'):
        with open(bp + fn, 'r') as f:
            raw_lines = f.readlines()
        sents = []
        raw_text = ''.join(raw_lines).replace('\xa0', ' ')
        raw_text = re.sub('\s+', ' ', raw_text).strip().replace('* ', '')
        
        for sent in list(razdel.sentenize(raw_text)):
            text = sent.text.replace('-\n', '').replace('\n', ' ').strip()
            if len(text) >= 3 and len(text) <= 500:
                sents.append(fix_text(text))
        fennougrica_sents.extend(sents)
        
print(len(fennougrica_sents))

54798


In [450]:
with open('other_sources/wikisource_myv_clean.json', 'r') as f:
    ws = json.load(f)

In [451]:
wikisource_sentences = []

for text in ws.values():
    sents = []
    raw_text = ''.join(text).replace('\xa0', ' ')
    raw_text = re.sub('\s+', ' ', raw_text).strip().replace('* ', '')

    for sent in list(razdel.sentenize(raw_text)):
        text = sent.text.replace('-\n', '').replace('\n', ' ').strip()
        if len(text) >= 3 and len(text) <= 500:
            sents.append(fix_text(text))
    wikisource_sentences.extend(sents)

print(len(wikisource_sentences))

120470


In [522]:
random.choice(wikisource_sentences)

'Секс, парсте-беряньстэ, содылинь Артёмонь ды Светань эрямосо валдо ды потмура читнеде.'

In [523]:
pravda = pd.read_csv('other_sources/erzya_pravda.tsv', sep='\t')

In [525]:
pravda_sents = []

for text in pravda.body:
    sents = []
    raw_text = ''.join(text).replace('\xa0', ' ')
    raw_text = re.sub('\s+', ' ', raw_text).strip().replace('* ', '')

    for sent in list(razdel.sentenize(raw_text)):
        text = sent.text.replace('-\n', '').replace('\n', ' ').strip()
        if len(text) >= 3 and len(text) <= 500:
            sents.append(fix_text(text))
    pravda_sents.extend(sents)
    
len(pravda_sents)

43772

In [548]:
random.choice(pravda_sents)

'Книганть пурнызе П.В. Рябов, витнемс тензэ лездасть Т.С. Баргова ды Т.П. Мокшанова (редакторт).'

In [549]:
with open('other_sources/livejournal_clean_myv.json', 'r') as f:
    lj = json.load(f)

In [550]:
lj_sentences = []

for text in tqdm(lj.values()):
    sents = []
    raw_text = ''.join(text).replace('\xa0', ' ')
    raw_text = re.sub('\s+', ' ', raw_text).strip().replace('* ', '')

    for sent in list(razdel.sentenize(raw_text)):
        text = sent.text.replace('-\n', '').replace('\n', ' ').strip()
        if len(text) >= 3 and len(text) <= 500:
            sents.append(fix_text(text))
    lj_sentences.extend(sents)

print(len(lj_sentences))

  0%|          | 0/1132 [00:00<?, ?it/s]

36584


In [551]:
with open('wiki/clean_myv.json', 'r') as f:
    wiki = json.load(f)

In [552]:
wiki_sentences = []

for text in wiki.values():
    sents = []
    raw_text = ''.join(text).replace('\xa0', ' ')
    raw_text = re.sub('\s+', ' ', raw_text).strip().replace('* ', '')

    for sent in list(razdel.sentenize(raw_text)):
        text = sent.text.replace('-\n', '').replace('\n', ' ').strip()
        if len(text) >= 3 and len(text) <= 500:
            sents.append(fix_text(text))
    wiki_sentences.extend(sents)

print(len(wiki_sentences))

59569


In [563]:
random.choice(wiki_sentences)

'Вирьга касы лембе-чивалгома ды куншкатаркань Азиява; кода пирень тикше касы эрьва-кува.'

In [566]:
erzya_bible = pd.read_csv('erzya_bible.tsv', sep='\t')
bible_sentences = erzya_bible.myv.dropna().apply(fix_text).tolist()
len(bible_sentences)

12912

In [571]:
random.choice(bible_sentences)

'Ды Мон тейса сынст марто те вейсэньлувонть, зярдо нардасынь пежетест».'

In [572]:
with open('other_sources/BrizhinskiyVasiliySergeevich_001___sentences_unaligned.json', 'r') as f:
    games = json.load(f)

In [573]:
games_sents = [p[0] for p in games['myv']]
print(len(games_sents))

5194


In [574]:
random.choice(games_sents)

'3-ц е т е й т е р е с ь.'

В итоге у меня набралось 300К уникальных предложений, что в общем-то весьма круто!

In [690]:
def replace_repeating_char(text, max_run=3):
    res = []
    prev = None
    run = 0
    for c in text:
        if c != prev:
            run = 0
        run += 1
        if run <= max_run:
            res.append(c)
        prev = c
    return ''.join(res)

replace_repeating_char("' 2768 [8_________________________ 2400 3 сядт, 4 дес. 6 един.")

"' 2768 [8___ 2400 3 сядт, 4 дес. 6 един."

In [691]:
def final_clean(text):
    text = fix_text(text)
    text = replace_repeating_char(text)
    if max(len(w) for w in text.split()) < 2:
        return ''
    
    if not re.match('.*[а-яё].*', text.lower()):
        return ''
    text = re.sub('(\. )+', '. ', text)
    
    return text.strip()

In [692]:
all_sentences = sorted({
    final_clean(s) for s in set(
        books_sents 
        + fennougrica_sents
        + wikisource_sentences 
        + pravda_sents 
        + lj_sentences 
        + wiki_sentences 
        + bible_sentences 
        + games_sents
    )
}.union({t[0] for t in train}).difference(devtest_er).difference({''}))
print(len(all_sentences))

333651


In [711]:
for _ in range(10):
    print(random.choice(all_sentences))
    print('------')

1931—1932 иетнесэ ульнесь татронь эйкакшонь журналонь редакторокс, конань нолдась ЦК ВЛКСМ.
------
Сдано в производство 9/VI-40 г. Подписано к печати 13/VIII-40 г. Формат бум.
------
Юткось аволь покш — вете вайгельпеть.
------
атна ломань
------
Те арсевкстэнть кармась содамо Паволонь сазоронзо цёра. Сон мольсь ушмокудов ды ёвтась теде лелянстэнь.
------
И сон аф содасы, кода синь явомс.
------
сюдома
------
пургондамс
------
Роботась мольсь вадрясто.
------
А кувать кортасть — кедьёнкс мельга тусть: Лия велетнень ланга кучокшность Кулянь пачтицят, бойка чийницят.
------


### todo: clean by perplexity 

In [694]:
with open('corpus/myv_mono.v1.json', 'w') as f:
    json.dump(all_sentences, f, indent=2, ensure_ascii=False)

# Counting

In [22]:
len(set(joint_phrasebook_pairs))

42266

In [23]:
all_dict_pairs = list(set(joint_phrasebook_pairs + dict1_pairs))
len(all_dict_pairs)

42340

In [30]:
extra_sents = []

punct = '.?!'
for m, r in joint_phrasebook_pairs:
    if m[-1] in punct and r[-1] in punct and (len(m.split()) > 1 and len(r.split()) > 1):
        extra_sents.append((m, r))
        continue
    if (len(m.split()) > 1 and len(r.split()) > 1):
        print(m)
        print(r)
        print()

мирденень максомс
выдать замуж

лисемс мирденень
выйти замуж

вайгелев гайтть
гласные звуки

чачома чи
день рождения

эйкакшонь кужо
детский сад

шумбра чокшнэ
добрый вечер

чугункань ки
железная дорога

педиця орма
заразная болезнь

чинь валгома
заход солнца

Од ие
Новый год

секс мекс
потому что

вайгельтеме гайтть
согласные звуки

каязь эйкакш
абортированный плод

 каянь эйде
абортированный плод

Вирень Ордаш веле
Ардашево Лесное 

Паксянь Ордаш веле
Ардашево Напольное 

ино кода
а то как же

кода ино
а то как же

ловонь ломань
баба снежная

эйдень баба
бабка повивальная 

Кече Пуло
Большая Медведица

Пелевень курцякс
Большая Медведица

Сисем Озякат
Большая Медведица

ярмаконь путовкс
вклад денежный

Раськень Озкс
Всеэрзянское моление

мирдес лисемс
выйти замуж

 аштема кудо
дом для посиделок

нупонь чей
камыш полый

кочкамонь кесак
клуб дыма

сувонь кесак
клуб пара

калонь сардо
кость рыбы

сэтьме эрямо тенк
мир вам

Каргонь Ки
Млечный путь

ава ловсо
молоко материнское

цят якшамо


лома велень ломань
чужой человек

лома велень ломань
человек из другого населённого пункта

ломанень кудо
чужой дом

ломань ютксо
на миру

ломань ютксо
в людях

латось лондадсь
крыша провалилась

лоськовтозь пиземс
ливмя лить

лосьсэ пиземс
лить ливнем

тикше лоштямс
прессовать сёно

тесэ лувозо
в зтом его сущность

лувозо а муеви
не доберёшься до его сущности

ал лукш
яичная скорлупа

катфа лукш
картофельная кожура

кал лукш
рыбья чешуя

Ленинэнь мавзолей
мавзолей Ленина

магнитсэ теке тарги
как-будто магнитом тянет

магнитэнь таргамо
магнитное притяжение

магнитэнь вий
магнитная сила, сила магнита

пакшась мадевсь
ребёнок улёгся

войнастонть ламо ломань мадевсть
во время войны много людей полегло

мадевтик эйкакшонть
уложи ребёнка

толось мадекшнесь
огонь потухал

виев пиземеденть сюротне мадекшнесть
от сильного дождя хлеба повалились

молян мадян
пойду лягу

толбандясь мадсь
костёр потух

чизэ мадсь
солнце погасло

маень чистэ
в майский день

мазый чама
красивое лицо

мазыйде мазый

рука засунута в карман

тонолыязь пенгть
обугленные дрова

задачась топавтозь
задача выполнена

план топавтан
план выполняю

пряс топавтомс
доверху наполни

кода пильгеэнзэ топади монь лангс
как топнет ногой на меня

топо да пиже понасо
густозелёного цвета, тёмно-зелёного цвета

топодекшнесть тензэ вете иеть
ему исполнилось десять лет

кирьгава топодят
по горло насытишься

корзинам топодсь
корзина моя наполнилась

цёрынентень топодсть кемень иеть
мальчику исполнилось десять лет

торговой тев
торговое дело

лопатнень пачк пиземесь тородеви минек лангс
сквозь листья дождь льёт на нас

сепараторонь пачк тородезь ловсо
процеженное через сепаратор молоко

пиземесь тородезь валсь
дождь ливмя лил

верде тороди пиземе
сверху льёт дождь

тороземезь сюро
набухшее зерно

кенкшесь летькенть эйстэ тороземсь
дверь от сырости набухла

шолксо торочазь панар
шёлком отороченная рубашка

ведесь тортав
вода мутная

потмакссо ансяк тортат кадовсть
на дне только осадки остались

торфонь таргамо
добывание то

In [31]:
print(len(extra_sents))

238


In [53]:
all_pairs = sorted({
    c for s in all_sources for c in s 
    if c[0] and c[1]
})
print(len(all_pairs))

77503


In [54]:
mined_pairs = sorted({
    c for s in all_sources[4:] for c in s 
    if c[0] and c[1]
})
print(len(mined_pairs))

21411


In [57]:
sent_pairs = sorted({
    c for s in all_sources[2:] for c in s 
    if c[0] and c[1]
})
print(len(sent_pairs))

35181


In [58]:
for _ in range(20):
    print(random.choice(sent_pairs))

('Минь эрьгелинек ды ракинек, минек кургонок эзть лотксе шнамонь морамодо. Сестэ лия раськетнень ютксо кортасть: «Азорось тейсь сыненст ине тев!».', "тогда уста наши были полны веселья, и язык наш–пения; тогда между народами говорили: `великое сотворил Господь над ними!'")
('Кратерэнь ванномань перть кой-кона геологтнэ кортасть седе, што паряк кратэрсэ улить алмазт, вадря покшолмасо, 0,03 кар ды седе покшт, Сынь теевкшныть взрывень шкане пек покш температурань ды лепштямонь кувалт.', 'В период интенсивного исследования кратера рядом геологов было высказано предположение о том, что в кратере могут присутствовать алмазы ювелирных или тезаврационных размеров — более 0,03 кар, образовавшиеся в момент взрыва под воздействием колоссальных температур и давления.')
('Иисус велявтсь фарисеенть ёнов ды мерсь тензэ: «Симон, Мон тонеть мезе-бути ёвтавлинь». Симон тень каршо пшкадсь: «Ёвтак, тонавтыця».', 'Обратившись к нему, Иисус сказал: Симон! Я имею нечто сказать тебе. Он говорит: скажи, Учител